In [5]:
import importlib
resources = importlib.import_module("resources")
import resources.MonteCarlo as MC

import numpy as np
import pandas as pd
import math
from functools import reduce
import random
random.seed(2010)

def cost_powers_k(rv, k = 2):
    if k == 2:
        cutoff = 1<<(int(rv)).bit_length()
    else:
        cutoff = k**(math.ceil(math.log(rv, k)))
    space_cost = cutoff-rv
    time_cost = (cutoff - 1)/(k-1)
    return (space_cost,time_cost)

def sum_pair_costs(sim, n,verbose = True):
    sum1,sq_sum1,sum2,sq_sum2 = reduce(lambda x,y : (x[0]+y[0], x[0]+y[0]**2,\
                                                     x[1]+y[1], x[1]+y[1]**2),\
                                        sim.sample_repeated(n),(0,0,0,0))
    if verbose:
        print("confidence for space complexity")
        MC.Calculate_ConfIntv(sum1,sq_sum1,n,.99)
        print("confidence for time complexity")
        MC.Calculate_ConfIntv(sum2,sq_sum2,n,.99)
    return (sum1, sq_sum1, sum2, sq_sum2)
    
alpha = -3

inv_f = lambda x : MC.inverse_continuous_power_law(x,1,alpha)
X = MC.RandomVariable(inv_f,"power law, alpha = {alph}, for x > 1".format(alph = alpha))
Sim = MC.Simulator(X,cost_powers_k,'Live-Graph')
n = 200000

sum_pair_costs(Sim,n)


testing random variable with distribution power law, alpha = -3, for x > 1,
 1.0777994721052366

New simulator for Live-Graph,
 with power law, alpha = -3, for x > 1 input
confidence for space complexity
We obtained a 0.99% confidence interval of
1.0032737929770559 +- (6.391295504392027e-14+1043.7777665922772j)
confidence for time complexity
We obtained a 0.99% confidence interval of
1.0032851500620643 +- (6.3913678541815e-14+1043.7895822095675j)


In [6]:
n = 2000000

    j = 10
    i = 2.1-j/100
    inv_f = lambda x : MC.inverse_continuous_power_law(x,1,-3)
    cost_f = lambda x : cost_powers_k(x,i)
    print ("\n i = {} \n".format(i))
    X = MC.RandomVariable(inv_f,"power law, alpha = -2, for x > 1",verbose = False)
    Sim = MC.Simulator(X,cost_f,'Live-Graph', verbose = False)
    sum_pair_costs(Sim,n, verbose = False)



 i = 2.0 

confidence for space complexity
We obtained a 0.99% confidence interval of
1.0031973910103344 +- (2.0209557190559625e-13+3300.4711570111976j)
confidence for time complexity
We obtained a 0.99% confidence interval of
1.0031974573073303 +- (2.0209558526122055e-13+3300.471375125095j)


In [19]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
%matplotlib inline

# Define the expression whose roots we want to find

alpha = -3
beta = .2

recurance_relationship = lambda p1,p2 : lambda p3: beta*p3**(alpha+1)/(alpha+1)-p3*p2**(alpha)  -\
                            (1-beta)*p2**(alpha+1)-p1**(alpha+1)/(alpha+1) + beta*(p1*p2**(alpha))
initial_condition = lambda p1: lambda p2: beta*p2**(alpha+1)/(alpha+1) - p2*p1**(alpha) - (1-beta)*p1**(alpha+1)-1.0/(alpha+1)


p1 = 1
func = initial_condition(p1)
initial_guess = 1.0/beta
p2 = fsolve(func, initial_guess)[0]
cutoffList = [p1,p2]

for i in range(10):
    func = recurance_relationship(p1,p2)
    initial_guess = p2/beta
    p3 = fsolve(func,initial_guess)[0]
    cutoffList.append(p3)
    p1 = p2
    p2 = p3


print (cutoffList)

[1, -0.5886211907610405, 0.6216920639901612, -0.4107450841611377, 0.40544312290349055, -0.2888583635120118, 0.2715504250230183, -0.20341401209305082, 0.18465506359352585, -0.1430791689687923, 0.12669173223907212, -0.10045438149476393]
